In [1]:
!pip install pytorch-widedeep

  Using cached pytorch_widedeep-1.2.2-py3-none-any.whl (21.1 MB)
     -------------------------------------- 617.1/617.1 kB 7.8 MB/s eta 0:00:00
  Using cached imutils-0.5.4.tar.gz (17 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached opencv_contrib_python-4.7.0.72-cp37-abi3-win_amd64.whl (44.9 MB)
     --------------------------------------- 12.2/12.2 MB 12.4 MB/s eta 0:00:00
  Using cached einops-0.6.1-py3-none-any.whl (42 kB)
     ---------------------------------------- 1.2/1.2 MB 5.8 MB/s eta 0:00:00
  Using cached torchmetrics-0.11.4-py3-none-any.whl (519 kB)
     --------------------------------------- 20.6/20.6 MB 10.9 MB/s eta 0:00:00
     ---------------------------------------- 1.2/1.2 MB 13.3 MB/s eta 0:00:00
  Using cached catalogue-2.0.8-py3-none-any.whl (17 kB)
     ---------------------------------------- 2.1/2.1 MB 12.3 MB/s eta 0:00:00
  Using cached spacy_loggers-1.0.4-py3-none-any.whl (11 kB)
   

In [2]:
import os
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import matplotlib.font_manager as fm
import matplotlib
import itertools


from itertools import product  
from math import sqrt
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from pytorch_widedeep.preprocessing import WidePreprocessor, TabPreprocessor
from pytorch_widedeep.models import Wide, TabMlp, WideDeep
from pytorch_widedeep.metrics import Accuracy
from pytorch_widedeep import Trainer

Matplotlib is building the font cache; this may take a moment.


In [3]:
FILES_DIR = '../data/a1/'
log_df = pd.read_csv(FILES_DIR + 'log_csv.csv')

In [4]:
# null값들을 un_** 으로 채워서 확인해보기
log_drop_null = log_df
log_drop_null['geoip_city_name'].fillna('un_city_name', inplace = True)
log_drop_null['category3'].fillna('un_category', inplace = True)
log_drop_null['uid'].fillna('un_name', inplace = True)
log_drop_null.dropna(axis = 0, inplace = True)
print(log_drop_null.isnull().sum().sort_values(ascending=False)) # null값 퍼센트로 표현

Unnamed: 0         0
uid                0
action_type        0
category1          0
category2          0
category3          0
collect_time       0
geoip_city_name    0
useragent_os       0
viewrate           0
viewtime           0
dtype: int64


In [5]:
log_drop_null['rating'] = log_drop_null['action_type']
re_name = {'rating' : {'View':1, 'Highlight':2, 'Basket':4, 'Copy':3, 'SINF':0}}
log_drop_null = log_drop_null.replace(re_name)
log_drop_null

,Unnamed: 0,uid,action_type,category1,category2,category3,collect_time,geoip_city_name,useragent_os,viewrate,viewtime,rating
0,0,avpKyWMn/xEADNlb,View,주방용품,그릇·홈세트,식판·나눔접시,2022-12-29 17:29:02,Seongnam-si,Windows,100.0,80033,1
1,1,0+5vEmGkOuAAB5j2,Basket,패브릭,매트,주방·다용도매트,2022-12-29 12:56:38,Yongin-si,Windows,0.0,0,4
2,2,r8DXy2OtSl8AAojG,View,패브릭,커튼·부자재,암막커튼,2022-12-29 17:11:30,Seoul,iOS,100.0,947,1
3,3,dC9Fl2Ly9lcADrTn,Highlight,데코·식물,플라워·식물,화병·화분,2022-12-29 17:54:30,Gwangmyeong-si,Windows,0.0,0,2
4,4,09gKcGLg0I8ACyQy,View,가구,진열장·책장,책장,2022-12-29 18:45:05,Jongno-gu,Mac OS X,100.0,1238,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5659536,5659536,PUpxWWMWdcIACR2B,View,주방용품,칼·커팅기구,채칼·다지기·강판,2022-12-28 20:01:30,Seoul,Android,100.0,13893,1
5659537,5659537,PVMstmNg4I4ADq9R,Basket,가구,진열장·책장,책장,2022-12-28 14:30:09,Jinju,Windows,0.0,0,4
5659538,5659538,diRXdmN49dUABrVo,View,가구,테이블·식탁·책상,식탁,2022-12-29 12:45:13,Gunpo,Android,100.0,47217,1
5659539,5659539,dusFzWJvp5cADhZW,View,가전·디지털,계절가전,전기히터·온풍기,2022-12-28 11:07:30,Seoul,Android,100.0,54030,1


In [6]:
# 날짜 데이터 데이트타임으로 바꾸기
def _trans_date_time(log):
    log.index = pd.to_datetime(log['collect_time'], infer_datetime_format=True)
    log.drop('collect_time', axis=1, inplace=True)
    return log

_trans_date_time(log_drop_null)

,Unnamed: 0,uid,action_type,category1,category2,category3,geoip_city_name,useragent_os,viewrate,viewtime,rating
collect_time,,,,,,,,,,,
2022-12-29 17:29:02,0,avpKyWMn/xEADNlb,View,주방용품,그릇·홈세트,식판·나눔접시,Seongnam-si,Windows,100.0,80033,1
2022-12-29 12:56:38,1,0+5vEmGkOuAAB5j2,Basket,패브릭,매트,주방·다용도매트,Yongin-si,Windows,0.0,0,4
2022-12-29 17:11:30,2,r8DXy2OtSl8AAojG,View,패브릭,커튼·부자재,암막커튼,Seoul,iOS,100.0,947,1
2022-12-29 17:54:30,3,dC9Fl2Ly9lcADrTn,Highlight,데코·식물,플라워·식물,화병·화분,Gwangmyeong-si,Windows,0.0,0,2
2022-12-29 18:45:05,4,09gKcGLg0I8ACyQy,View,가구,진열장·책장,책장,Jongno-gu,Mac OS X,100.0,1238,1
...,...,...,...,...,...,...,...,...,...,...,...
2022-12-28 20:01:30,5659536,PUpxWWMWdcIACR2B,View,주방용품,칼·커팅기구,채칼·다지기·강판,Seoul,Android,100.0,13893,1
2022-12-28 14:30:09,5659537,PVMstmNg4I4ADq9R,Basket,가구,진열장·책장,책장,Jinju,Windows,0.0,0,4
2022-12-29 12:45:13,5659538,diRXdmN49dUABrVo,View,가구,테이블·식탁·책상,식탁,Gunpo,Android,100.0,47217,1


In [7]:
log_drop_null["Hour"] = log_drop_null.index.hour

In [8]:
log_drop_null

,Unnamed: 0,uid,action_type,category1,category2,category3,geoip_city_name,useragent_os,viewrate,viewtime,rating,Hour
collect_time,,,,,,,,,,,,
2022-12-29 17:29:02,0,avpKyWMn/xEADNlb,View,주방용품,그릇·홈세트,식판·나눔접시,Seongnam-si,Windows,100.0,80033,1,17
2022-12-29 12:56:38,1,0+5vEmGkOuAAB5j2,Basket,패브릭,매트,주방·다용도매트,Yongin-si,Windows,0.0,0,4,12
2022-12-29 17:11:30,2,r8DXy2OtSl8AAojG,View,패브릭,커튼·부자재,암막커튼,Seoul,iOS,100.0,947,1,17
2022-12-29 17:54:30,3,dC9Fl2Ly9lcADrTn,Highlight,데코·식물,플라워·식물,화병·화분,Gwangmyeong-si,Windows,0.0,0,2,17
2022-12-29 18:45:05,4,09gKcGLg0I8ACyQy,View,가구,진열장·책장,책장,Jongno-gu,Mac OS X,100.0,1238,1,18
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-28 20:01:30,5659536,PUpxWWMWdcIACR2B,View,주방용품,칼·커팅기구,채칼·다지기·강판,Seoul,Android,100.0,13893,1,20
2022-12-28 14:30:09,5659537,PVMstmNg4I4ADq9R,Basket,가구,진열장·책장,책장,Jinju,Windows,0.0,0,4,14
2022-12-29 12:45:13,5659538,diRXdmN49dUABrVo,View,가구,테이블·식탁·책상,식탁,Gunpo,Android,100.0,47217,1,12


In [9]:
log = log_drop_null.drop_duplicates()
log

,Unnamed: 0,uid,action_type,category1,category2,category3,geoip_city_name,useragent_os,viewrate,viewtime,rating,Hour
collect_time,,,,,,,,,,,,
2022-12-29 17:29:02,0,avpKyWMn/xEADNlb,View,주방용품,그릇·홈세트,식판·나눔접시,Seongnam-si,Windows,100.0,80033,1,17
2022-12-29 12:56:38,1,0+5vEmGkOuAAB5j2,Basket,패브릭,매트,주방·다용도매트,Yongin-si,Windows,0.0,0,4,12
2022-12-29 17:11:30,2,r8DXy2OtSl8AAojG,View,패브릭,커튼·부자재,암막커튼,Seoul,iOS,100.0,947,1,17
2022-12-29 17:54:30,3,dC9Fl2Ly9lcADrTn,Highlight,데코·식물,플라워·식물,화병·화분,Gwangmyeong-si,Windows,0.0,0,2,17
2022-12-29 18:45:05,4,09gKcGLg0I8ACyQy,View,가구,진열장·책장,책장,Jongno-gu,Mac OS X,100.0,1238,1,18
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-28 20:01:30,5659536,PUpxWWMWdcIACR2B,View,주방용품,칼·커팅기구,채칼·다지기·강판,Seoul,Android,100.0,13893,1,20
2022-12-28 14:30:09,5659537,PVMstmNg4I4ADq9R,Basket,가구,진열장·책장,책장,Jinju,Windows,0.0,0,4,14
2022-12-29 12:45:13,5659538,diRXdmN49dUABrVo,View,가구,테이블·식탁·책상,식탁,Gunpo,Android,100.0,47217,1,12


In [10]:
log_sample = log.sample(frac=0.01, random_state=42)
log_sample = log_sample[~log_sample.index.duplicated(keep='first')]
log_sample

,Unnamed: 0,uid,action_type,category1,category2,category3,geoip_city_name,useragent_os,viewrate,viewtime,rating,Hour
collect_time,,,,,,,,,,,,
2022-12-27 14:39:16,3595088,bgojimOqhQEACbYY,View,가전·디지털,휴대폰,휴대폰,Seoul,Android,100.0,3006,1,14
2022-12-26 10:16:23,966238,0zPC5WOId8YABX08,Copy,가전·디지털,계절가전,전기요·온수매트,Yeongdeungpo-gu,Mac OS X,0.0,0,3,10
2022-12-26 13:08:35,2237030,t2umo2IVzewAAJiY,Highlight,패브릭,홈패브릭,홈웨어,Daejeon,Windows,0.0,0,2,13
2022-12-24 03:11:27,868149,AeM3zWOgVs4ABfZU,View,패브릭,러그·카페트,극세사·단모러그,Yongin-si,iOS,100.0,3906,1,3
2022-12-30 13:39:20,4828400,cJLHnmOuWO4AA50D,View,노하우,생활정보,un_category,Seongdong-gu,Mac OS X,100.0,44363,1,13
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-28 17:05:46,1821592,c1+lJGN3POgADwNi,View,가구,거울,전신거울,un_city_name,Windows,100.0,46759,1,17
2022-12-30 12:33:24,2919103,eaIjAmOuWoEACNHp,Highlight,가구,침대,침대프레임,Mapo-gu,Windows,0.0,0,2,12
2022-12-28 19:13:40,2997600,dtqrEmJUEyMADjG0,View,가구,선반,스탠드선반,Gunsan,Windows,0.0,0,1,19


In [11]:
log_uid = log_sample[['uid', 'action_type', 'viewrate', 'viewtime', 'rating', 'Hour']]
log_uid

,uid,action_type,viewrate,viewtime,rating,Hour
collect_time,,,,,,
2022-12-27 14:39:16,bgojimOqhQEACbYY,View,100.0,3006,1,14
2022-12-26 10:16:23,0zPC5WOId8YABX08,Copy,0.0,0,3,10
2022-12-26 13:08:35,t2umo2IVzewAAJiY,Highlight,0.0,0,2,13
2022-12-24 03:11:27,AeM3zWOgVs4ABfZU,View,100.0,3906,1,3
2022-12-30 13:39:20,cJLHnmOuWO4AA50D,View,100.0,44363,1,13
...,...,...,...,...,...,...
2022-12-28 17:05:46,c1+lJGN3POgADwNi,View,100.0,46759,1,17
2022-12-30 12:33:24,eaIjAmOuWoEACNHp,Highlight,0.0,0,2,12
2022-12-28 19:13:40,dtqrEmJUEyMADjG0,View,0.0,0,1,19


In [12]:
train_df, test_df = train_test_split(log_uid, test_size=0.2, random_state=42)

print(train_df.shape)
print(test_df.shape)

(42901, 6)
(10726, 6)


In [13]:
train_df

,uid,action_type,viewrate,viewtime,rating,Hour
collect_time,,,,,,
2022-12-28 15:14:35,fG9bBGLLvU8AAhSw,View,100.0,33229,1,15
2022-12-28 15:47:58,2pHmumMiuk8AA15U,View,100.0,6558,1,15
2022-12-27 15:43:05,cJv7YmOqdsYABmgZ,View,100.0,7113,1,15
2022-12-29 18:01:23,2pxI+GA26uIABMoQ,View,100.0,48488,1,18
2022-12-27 15:39:55,2ia/BWGAy88ACEXT,Highlight,0.0,0,2,15
...,...,...,...,...,...,...
2022-12-29 10:48:37,PUrdQGLwkoMADK5q,View,100.0,38359,1,10
2022-12-27 16:21:15,ebVcK2BRsa8ABaR8,View,100.0,1030,1,16
2022-12-26 16:34:48,diHkOWJwkoYABHbU,View,100.0,7015,1,16


In [14]:
category1 = [(list(set(x['uid'].values))[0], '/'.join(x['category1'].values)) for index, x in log_sample.groupby('uid')]
category1_df = pd.DataFrame(data=category1, columns=['uid', 'category1'])
category1_df = category1_df.set_index('uid')


category2 = [(list(set(x['uid'].values))[0], '/'.join(x['category2'].values)) for index, x in log_sample.groupby('uid')]
category2_df = pd.DataFrame(data=category2, columns=['uid', 'category2'])
category2_df = category2_df.set_index('uid')


countries = [(list(set(x['uid'].values))[0], ','.join(x['geoip_city_name'].values)) for index, x in log_sample.groupby('uid')]
countries_df = pd.DataFrame(data=countries, columns=['uid', 'country'])
countries_df = countries_df.set_index('uid')


os = [(list(set(x['uid'].values))[0], ','.join(x['useragent_os'].values)) for index, x in log_sample.groupby('uid')]
os_df = pd.DataFrame(data=os, columns=['uid', 'OS'])
os_df = os_df.set_index('uid')



log_total = pd.concat([log_uid, category1_df, category2_df, countries_df, os_df], axis=1)

print(log_total.shape)
print(log_total.head())

(92611, 10)
                                  uid action_type  viewrate  viewtime  rating  \
2022-12-27 14:39:16  bgojimOqhQEACbYY        View     100.0    3006.0     1.0   
2022-12-26 10:16:23  0zPC5WOId8YABX08        Copy       0.0       0.0     3.0   
2022-12-26 13:08:35  t2umo2IVzewAAJiY   Highlight       0.0       0.0     2.0   
2022-12-24 03:11:27  AeM3zWOgVs4ABfZU        View     100.0    3906.0     1.0   
2022-12-30 13:39:20  cJLHnmOuWO4AA50D        View     100.0   44363.0     1.0   

                     Hour category1 category2 country   OS  
2022-12-27 14:39:16  14.0       NaN       NaN     NaN  NaN  
2022-12-26 10:16:23  10.0       NaN       NaN     NaN  NaN  
2022-12-26 13:08:35  13.0       NaN       NaN     NaN  NaN  
2022-12-24 03:11:27   3.0       NaN       NaN     NaN  NaN  
2022-12-30 13:39:20  13.0       NaN       NaN     NaN  NaN  


In [15]:
dummy_cat1_df = log_total['category1'].str.get_dummies(sep='/')
train_df_cat1 = train_df['uid'].apply(lambda x: dummy_cat1_df.loc[x])

dummy_cat2_df = log_total['category2'].str.get_dummies(sep='/')
train_df_cat2 = train_df['uid'].apply(lambda x: dummy_cat2_df.loc[x])

dummy_country_df = log_total['country'].str.get_dummies(sep=',')
train_df_country = train_df['uid'].apply(lambda x: dummy_country_df.loc[x])

dummy_os_df = log_total['OS'].str.get_dummies(sep=',')
train_df_OS = train_df['uid'].apply(lambda x: dummy_os_df.loc[x])

In [16]:
train_df = pd.concat([train_df, train_df_cat1, train_df_cat2, train_df_country, train_df_OS], axis=1)
train_df = train_df.set_index(['uid'])
train_df.head()

,action_type,viewrate,viewtime,rating,Hour,(주)씨밀렉스코리아,1401home,2uc,3M,45팩토리,...,Android,CentOS,Chrome OS,Linux,Mac OS X,Other,Tizen,Ubuntu,Windows,iOS
uid,,,,,,,,,,,,,,,,,,,,,
fG9bBGLLvU8AAhSw,View,100.0,33229,1,15,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2pHmumMiuk8AA15U,View,100.0,6558,1,15,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
cJv7YmOqdsYABmgZ,View,100.0,7113,1,15,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2pxI+GA26uIABMoQ,View,100.0,48488,1,18,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2ia/BWGAy88ACEXT,Highlight,0.0,0,2,15,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [17]:
wide_cols = list(train_df_cat1.columns) + list(train_df_cat2.columns) + list(train_df_country.columns) + list(train_df_OS.columns)
wide_cols

['(주)씨밀렉스코리아',
 '1401home',
 '2uc',
 '3M',
 '45팩토리',
 'AMT',
 'DDWD',
 'DELI',
 'DF동서가구',
 'Decor Plant',
 'Ev Aletleri Dijital',
 'FREESPACE',
 'GALON (갈롱)',
 'GDK',
 'HAEGANG',
 'HAILY(헤일리)',
 'HICKIES',
 'Herman Miller',
 'Home Appliances Digital',
 'Household goods',
 'JAJU(자주)',
 'Know-how ',
 'LG전자',
 'LX지인',
 'MF매직하우스',
 'MJK',
 'MOD가구',
 'MURMUR',
 'NHB',
 'OMT',
 'PN풍년',
 'Rome&Glas',
 'Samsung',
 'Storage, organization',
 'THE살림',
 'Thông ',
 'Tool, DIY',
 'UFO',
 'VOHE',
 'YLB예림방',
 'bsf',
 'fabric',
 'furniture',
 'kitchen utensils',
 'light',
 'mebel',
 'sk매직',
 'vitra',
 'デコ・植物',
 'ファブリック',
 '光',
 '厨房用具',
 '家具',
 '家电 数码',
 '生活用品',
 '가구',
 '가구느낌',
 '가구의가치',
 '가본조명',
 '가온퍼니처',
 '가이온',
 '가전',
 '가전·디지털',
 '감성다는남자',
 '강블리라이프',
 '고그린오브젝트',
 '고르소',
 '공간LED',
 '공간디자인',
 '공구 DIY',
 '공구·DIY',
 '공방 오롯이',
 '구스퍼니처',
 '글로잉컬러',
 '금토일샵',
 '김성훈도자기',
 '김용석생활연구소',
 '까르데코',
 '까르엠가구',
 '까사마루',
 '까사무띠',
 '까사미아',
 '까사베르데',
 '꾸미까',
 '꾸민뜰',
 '끌레오',
 '나고미도자기',
 '나누벨',
 '나무뜰',
 '네스홈',
 '네오플램',
 '네이

In [18]:
# 리소스 문제로 5개만
wide_cols = wide_cols[:5]
wide_cols

['(주)씨밀렉스코리아', '1401home', '2uc', '3M', '45팩토리']

In [19]:
unique_combinations = list(list(zip(wide_cols, element)) 
                           for element in product(wide_cols, repeat = len(wide_cols))) 

print(unique_combinations)
cross_cols = [item for sublist in unique_combinations for item in sublist]
cross_cols = [x for x in cross_cols if x[0] != x[1]]
cross_cols = list(set(cross_cols))
print(cross_cols)

[[('(주)씨밀렉스코리아', '(주)씨밀렉스코리아'), ('1401home', '(주)씨밀렉스코리아'), ('2uc', '(주)씨밀렉스코리아'), ('3M', '(주)씨밀렉스코리아'), ('45팩토리', '(주)씨밀렉스코리아')], [('(주)씨밀렉스코리아', '(주)씨밀렉스코리아'), ('1401home', '(주)씨밀렉스코리아'), ('2uc', '(주)씨밀렉스코리아'), ('3M', '(주)씨밀렉스코리아'), ('45팩토리', '1401home')], [('(주)씨밀렉스코리아', '(주)씨밀렉스코리아'), ('1401home', '(주)씨밀렉스코리아'), ('2uc', '(주)씨밀렉스코리아'), ('3M', '(주)씨밀렉스코리아'), ('45팩토리', '2uc')], [('(주)씨밀렉스코리아', '(주)씨밀렉스코리아'), ('1401home', '(주)씨밀렉스코리아'), ('2uc', '(주)씨밀렉스코리아'), ('3M', '(주)씨밀렉스코리아'), ('45팩토리', '3M')], [('(주)씨밀렉스코리아', '(주)씨밀렉스코리아'), ('1401home', '(주)씨밀렉스코리아'), ('2uc', '(주)씨밀렉스코리아'), ('3M', '(주)씨밀렉스코리아'), ('45팩토리', '45팩토리')], [('(주)씨밀렉스코리아', '(주)씨밀렉스코리아'), ('1401home', '(주)씨밀렉스코리아'), ('2uc', '(주)씨밀렉스코리아'), ('3M', '1401home'), ('45팩토리', '(주)씨밀렉스코리아')], [('(주)씨밀렉스코리아', '(주)씨밀렉스코리아'), ('1401home', '(주)씨밀렉스코리아'), ('2uc', '(주)씨밀렉스코리아'), ('3M', '1401home'), ('45팩토리', '1401home')], [('(주)씨밀렉스코리아', '(주)씨밀렉스코리아'), ('1401home', '(주)씨밀렉스코리아'), ('2uc', '(주)씨밀렉스코리아'), ('3M', '1401home'), ('45팩토리', '2uc'

In [20]:
embed_cols = list(set([(x[0], 16) for x in cross_cols]))
continuous_cols = ['Hour']

print(embed_cols)
print(continuous_cols)

[('45팩토리', 16), ('1401home', 16), ('(주)씨밀렉스코리아', 16), ('3M', 16), ('2uc', 16)]
['Hour']


In [21]:
target = train_df['rating']

In [22]:
preprocess_wide = WidePreprocessor(wide_cols=wide_cols, crossed_cols=cross_cols)
X_wide = preprocess_wide.fit_transform(train_df)
wide = Wide(input_dim=np.unique(X_wide).shape[0], pred_dim=1)

In [23]:
X_wide.size

1072525

In [24]:
preprocess_tap = TabPreprocessor(embed_cols=embed_cols, continuous_cols=continuous_cols)
X_tab = preprocess_tap.fit_transform(train_df)
tabmlp = TabMlp(
    column_idx=preprocess_tap.column_idx,
    cat_embed_input=preprocess_tap.cat_embed_input,
    continuous_cols=continuous_cols,
)

In [25]:
tabmlp

TabMlp(
  (cat_and_cont_embed): DiffSizeCatAndContEmbeddings(
    (cat_embed): DiffSizeCatEmbeddings(
      (embed_layers): ModuleDict(
        (emb_layer_45팩토리): Embedding(3, 16, padding_idx=0)
        (emb_layer_1401home): Embedding(3, 16, padding_idx=0)
        (emb_layer_(주)씨밀렉스코리아): Embedding(2, 16, padding_idx=0)
        (emb_layer_3M): Embedding(3, 16, padding_idx=0)
        (emb_layer_2uc): Embedding(3, 16, padding_idx=0)
      )
      (embedding_dropout): Dropout(p=0.1, inplace=False)
    )
    (cont_norm): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (encoder): MLP(
    (mlp): Sequential(
      (dense_layer_0): Sequential(
        (0): Dropout(p=0.1, inplace=False)
        (1): Linear(in_features=81, out_features=200, bias=True)
        (2): ReLU(inplace=True)
      )
      (dense_layer_1): Sequential(
        (0): Dropout(p=0.1, inplace=False)
        (1): Linear(in_features=200, out_features=100, bias=True)
        (2): ReLU(inplace=T

In [26]:
model = WideDeep(wide=wide, deeptabular=tabmlp)

# train and validate
trainer = Trainer(model, objective="binary", metrics=[Accuracy])
trainer.fit(
    X_wide=X_wide,
    X_tab=X_tab,
    target=target,
    n_epochs=5,
    batch_size=64,
)

epoch 5: 100%|███████████████████████████████| 671/671 [01:37<00:00,  6.90it/s, loss=-1.65e+8, metrics={'acc': 0.8711}]


In [27]:
X_wide_te = wide_preprocessor.transform(test_df)
X_tab_te = tab_preprocessor.transform(test_df)
preds = trainer.predict(X_wide=X_wide_te, X_tab=X_tab_te)

NameError: name 'wide_preprocessor' is not defined

In [ ]:
trainer.save(path="model_weights", save_state_dict=True)


torch.save(model.state_dict(), "model_weights/wd_model.pt")


model_new = WideDeep(wide=wide, deeptabular=tab_mlp)
model_new.load_state_dict(torch.load("model_weights/wd_model.pt"))


trainer_new = Trainer(model_new, objective="binary")


preds = trainer_new.predict(X_wide=X_wide, X_tab=X_tab)